# Objetivo

  Recolectar información de las APIS públicas de MercadoLibre (https://developers.mercadolibre.com.ar/es_ar/api-docs-es) para luego realizar un análisis de las publicaciones con descuentos.

  El análisis se acotará a Argentina teniendo en cuenta aproximadamente 1000 publicaciones por cada una de las categorías presentes. 

  Finalmente, se guardará la información recolectada en un archivo en formato .csv.



# Importanciones


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%cd "drive/MyDrive/DataScienceChallengeMELI/Análisis"

[Errno 2] No such file or directory: 'drive/MyDrive/DataScienceChallengeMELI/Análisis'
/content/drive/MyDrive/DataScienceChallengeMELI/Análisis


In [29]:
!pip install import_ipynb

In [30]:
import numpy as np
import pandas as pd

import time
from time import process_time

import import_ipynb
from funciones_aux import request_a_diccionario, generar_dfs

# Recolección de información

## Países que tienen acceso a MercadoLibre

In [31]:
lista_paises = request_a_diccionario("https://api.mercadolibre.com/sites")

In [32]:
lista_paises

[{'default_currency_id': 'COP', 'id': 'MCO', 'name': 'Colombia'},
 {'default_currency_id': 'USD', 'id': 'MSV', 'name': 'El Salvador'},
 {'default_currency_id': 'PAB', 'id': 'MPA', 'name': 'Panamá'},
 {'default_currency_id': 'CLP', 'id': 'MLC', 'name': 'Chile'},
 {'default_currency_id': 'PEN', 'id': 'MPE', 'name': 'Perú'},
 {'default_currency_id': 'HNL', 'id': 'MHN', 'name': 'Honduras'},
 {'default_currency_id': 'BRL', 'id': 'MLB', 'name': 'Brasil'},
 {'default_currency_id': 'GTQ', 'id': 'MGT', 'name': 'Guatemala'},
 {'default_currency_id': 'CRC', 'id': 'MCR', 'name': 'Costa Rica'},
 {'default_currency_id': 'PYG', 'id': 'MPY', 'name': 'Paraguay'},
 {'default_currency_id': 'NIO', 'id': 'MNI', 'name': 'Nicaragua'},
 {'default_currency_id': 'DOP', 'id': 'MRD', 'name': 'Dominicana'},
 {'default_currency_id': 'UYU', 'id': 'MLU', 'name': 'Uruguay'},
 {'default_currency_id': 'USD', 'id': 'MEC', 'name': 'Ecuador'},
 {'default_currency_id': 'VES', 'id': 'MLV', 'name': 'Venezuela'},
 {'default_cu

## Categorías que tiene el sitio

In [33]:
lista_categorias = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/categories")

In [34]:
print("Hay un total de", len(lista_categorias), "categorías.")

Hay un total de 32 categorías.


In [35]:
lista_categorias

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

## Subcategorías que tiene el sitio

In [36]:
dicc_subcategorias = {}
for categoria in lista_categorias:
    categoria_id = categoria["id"]
    categoria_name = categoria["name"]
    info = request_a_diccionario("https://api.mercadolibre.com/categories/" + categoria_id)
    print("Categoría:", categoria_name)
    children_categories = info["children_categories"]
    print("Cantidad de subcategorías:", len(children_categories))
    for subcategoria in children_categories:
        dicc_subcategorias[subcategoria["id"]] = subcategoria["name"]
    print("-"*100)

Categoría: Accesorios para Vehículos
Cantidad de subcategorías: 22
----------------------------------------------------------------------------------------------------
Categoría: Agro
Cantidad de subcategorías: 8
----------------------------------------------------------------------------------------------------
Categoría: Alimentos y Bebidas
Cantidad de subcategorías: 7
----------------------------------------------------------------------------------------------------
Categoría: Animales y Mascotas
Cantidad de subcategorías: 10
----------------------------------------------------------------------------------------------------
Categoría: Antigüedades y Colecciones
Cantidad de subcategorías: 9
----------------------------------------------------------------------------------------------------
Categoría: Arte, Librería y Mercería
Cantidad de subcategorías: 5
----------------------------------------------------------------------------------------------------
Categoría: Autos, Motos y Ot

In [37]:
print("Hay un total de", len(dicc_subcategorias), "subcategorías.")

Hay un total de 409 subcategorías.


In [38]:
dicc_subcategorias

{'MLA1002': 'Televisores',
 'MLA10056': 'Souvenirs',
 'MLA100778': 'Kits Imprimibles para Fiestas',
 'MLA1015': 'Video',
 'MLA10372': 'Revelado y Laboratorio',
 'MLA10405': 'Recitales',
 'MLA10434': 'Eventos Deportivos',
 'MLA10451': 'Teatro',
 'MLA1049': 'Accesorios para Cámaras',
 'MLA10512': 'Entradas de Colección',
 'MLA10513': 'Otras Entradas',
 'MLA10514': 'Belleza y Cuidado Personal',
 'MLA105179': 'PH',
 'MLA1053': 'Telefonía Fija e Inalámbrica',
 'MLA1055': 'Celulares y Smartphones',
 'MLA1058': 'Handies y Radiofrecuencia',
 'MLA10616': 'Tarifadores y Cabinas',
 'MLA1070': 'Otros',
 'MLA1072': 'Perros',
 'MLA1081': 'Gatos',
 'MLA109026': 'Calzado',
 'MLA109042': 'Remeras, Musculosas y Chombas',
 'MLA109089': 'Trajes',
 'MLA1091': 'Peces',
 'MLA109254': 'Enteritos',
 'MLA109276': 'Bermudas y Shorts',
 'MLA109278': 'Ropa Interior y de Dormir',
 'MLA109282': 'Pantalones, Jeans y Joggings',
 'MLA10934': 'Mercería',
 'MLA1100': 'Aves',
 'MLA1105': 'Roedores',
 'MLA110761': 'Equipaj

## Recolección de datos 

In [39]:
t_inicial = time.perf_counter()

lista_dfs = []
for numero_categoria, categoria in enumerate(lista_categorias):
    print("Recolectando data de la categoría:", str(numero_categoria), "de", len(lista_categorias), "(categoría", categoria["name"], ").")
    for pagina in range(20):
        print("Página:" + str(pagina) + " de 19.", end='\r', flush=True)
        categoria_id = categoria["id"]
        categoria_name = categoria["name"]
        offset = str(50 * pagina + 1)
        items = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/search?category=" + categoria_id + "&limit=50&offset=" + offset)
        df = generar_dfs(items)
        df["categoria_principal"] = categoria_name
        lista_dfs.append(df)
        time.sleep(1)
    elapsed_time = time.perf_counter() - t_inicial
    print("Tiempo de ejecución:", round(elapsed_time / 60, 2), "minutos.")
    print("-"*100)
        
elapsed_time = time.perf_counter() - t_inicial

print("Tiempo total de ejecución:", round(elapsed_time / 60, 2), "minutos.")

Recolectando data de la categoría: 0 de 32 (categoría Accesorios para Vehículos ).
Tiempo de ejecución: 0.96 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 1 de 32 (categoría Agro ).
Tiempo de ejecución: 1.49 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 2 de 32 (categoría Alimentos y Bebidas ).
Tiempo de ejecución: 2.01 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 3 de 32 (categoría Animales y Mascotas ).
Tiempo de ejecución: 2.53 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 4 de 32 (categoría Antigüedades y Colecciones ).
Tiempo de ejecución: 3.05 minutos.
--------------------------------------

Se arma el dataframe

In [40]:
df_total_original = pd.concat(lista_dfs, axis = 0)

Se eliminan los duplicados

In [41]:
print("Cantidad de duplicados:", df_total_original.duplicated(keep="first").sum())
df_total_original = df_total_original[~df_total_original.duplicated(keep="first")]
print("Eliminamos los duplicados.")
print("Quedaron", len(df_total_original), "registros en total.")

Cantidad de duplicados: 100
Eliminamos los duplicados.
Quedaron 31789 registros en total.


Generación del archivo .csv

In [42]:
df_total_original.to_csv("df_total_original.csv", index = False)